In [2]:
import tensorflow as tf
import pathlib
import PIL
import numpy as np

import matplotlib
matplotlib.use("TKAgg")
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms
from PIL import Image
import os
import scipy

AUTOTUNE = tf.data.AUTOTUNE # prompts the tf.data runtime to tune the value dynamically at runtime



In [2]:
batch_size = 32
img_height = 28
img_width = 28

def create_dataset_from_path(path):

    # function that converts a file path to an (img, label) pair
    def get_label(file_path):
      # Convert the path to a list of path components
      parts = tf.strings.split(file_path, os.path.sep)
      # The second to last is the class-directory
      one_hot = parts[-2] == class_names
      # Integer encode the label
      return tf.argmax(one_hot)

    def decode_img(img):
      # Convert the compressed string to a 3D uint8 tensor
      img = tf.io.decode_jpeg(img, channels=1)
      # Resize the image to the desired size
      return tf.image.resize(img, [img_height, img_width])

    def process_path(file_path):
      label = get_label(file_path)
      # Load the raw data from the file as a string
      img = tf.io.read_file(file_path)
      img = decode_img(img)
      return img, label

    # Make the imgs in to batches
    def configure_for_performance(ds):
      ds = ds.cache()
      ds = ds.shuffle(buffer_size=1000)
      ds = ds.batch(batch_size)
      ds = ds.prefetch(buffer_size=AUTOTUNE)
      return ds

    data_dir = pathlib.Path(path) # for pycharm
    #data_dir = pathlib.Path('mnist_png/training') # for the jupyter notebook

    print(path+'\\*\\*')

    # Shuffle might be better set to False but this seems better for now
    list_ds = tf.data.Dataset.list_files((path+'\\*\\*'), shuffle=True)
    image_count = len(list_ds)
    class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
    print(class_names)
    val_size = int(image_count * 0.2)
    train_ds = list_ds.skip(val_size) # skips x initial elements from this dataset.
    val_ds = list_ds.take(val_size)
    print(tf.data.experimental.cardinality(train_ds).numpy())
    print(tf.data.experimental.cardinality(val_ds).numpy())


    # Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
    train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
    val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

    normalization_layer = tf.keras.layers.Rescaling(1./255)

    normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    # normalized_ds  -> range goes from 0 to 1
    # train_ds  -> range goes from 0 to 255

    normalized_ds = configure_for_performance(normalized_ds)
    val_ds = configure_for_performance(val_ds)

    print(normalized_ds)
    return normalized_ds, val_ds, class_names

In [3]:
#normalized_ds, val_ds = create_dataset_from_path('Classifier\\mnist_png\\training')
normalized_ds, val_ds, class_names = create_dataset_from_path('mnist_png\\me')


mnist_png\me\*\*
['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']
200
50
<PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [4]:
# Using the normalized
for imageBatch, labelBatch in normalized_ds.take(1):
    print("Image shape: ", imageBatch.numpy()[0].shape)
    print("Label: ", labelBatch.numpy()[0])
    print("Label: ", labelBatch.numpy())
    #print(np.min(image), np.max(image))

    # f = open("demofile.txt", "a")
    # for a in range(28): # goes from 0 to 27
    #   for b in range(28):
    #     print(f'{round(float(image[b][a][0]), 1)}',end='')
    #     f.write(str(round(float(image[b][a][0]), 1)))
    #   print(f'\n')
    #   f.write('\n')
    # f.close()

    imgToBeDisplayed = imageBatch.numpy()[0]

    extraChannel = np.zeros((28,28,1))
    imgToBeDisplayed = np.concatenate((imgToBeDisplayed,extraChannel), axis=2)
    imgToBeDisplayed = np.concatenate((imgToBeDisplayed,extraChannel), axis=2)
    imgToBeDisplayed = imgToBeDisplayed * 255

    print("Image shape: ", imgToBeDisplayed.shape)
    print("Label: ", labelBatch.numpy()[0])
    # #print(f'->{image}')

    print(np.min(imageBatch[0]), np.max(imageBatch[0]))

    image_uint8 = imgToBeDisplayed.astype(np.uint8)
    img = Image.fromarray(image_uint8, 'RGB')
    #img.show()
    img.save('Classifier\\temp.png')

PIL.Image.open('Classifier\\temp.png')


Image shape:  (28, 28, 1)
Label:  1
Label:  [1 5 4 3 9 0 4 2 4 0 7 9 2 5 3 4 7 3 7 4 1 3 1 4 2 3 6 8 9 1 6 7]
Image shape:  (28, 28, 3)
Label:  1
0.0 1.0


FileNotFoundError: [Errno 2] No such file or directory: 'Classifier\\temp.png'

In [ ]:
os.remove('Classifier/temp.png')


In [4]:
num_classes = 10

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(filters=15, kernel_size=(5,5), padding='Same', activation='relu', input_shape=(28, 28, 1),
                         data_format="channels_last"),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

  tf.keras.layers.Conv2D(filters=20, kernel_size=(10,10), padding='Same', activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

  tf.keras.layers.Conv2D(filters=25, kernel_size=(15,15), padding='Same', activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [5]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


In [6]:
epochs=10

history = model.fit(
  normalized_ds,
  validation_data=val_ds,
  epochs=epochs
)



Epoch 1/10
7/7 [==============================] - 6s 141ms/step - loss: 2.3068 - accuracy: 0.1450 - val_loss: 15.0923 - val_accuracy: 0.2200
Epoch 2/10
7/7 [==============================] - 0s 23ms/step - loss: 2.2017 - accuracy: 0.3050 - val_loss: 48.2233 - val_accuracy: 0.2400
Epoch 3/10
7/7 [==============================] - 0s 27ms/step - loss: 1.8878 - accuracy: 0.3650 - val_loss: 28.4725 - val_accuracy: 0.5800
Epoch 4/10
7/7 [==============================] - 0s 26ms/step - loss: 1.2893 - accuracy: 0.6700 - val_loss: 82.9771 - val_accuracy: 0.7200
Epoch 5/10
7/7 [==============================] - 0s 26ms/step - loss: 0.9291 - accuracy: 0.7000 - val_loss: 64.5850 - val_accuracy: 0.7000
Epoch 6/10
7/7 [==============================] - 0s 27ms/step - loss: 0.6890 - accuracy: 0.7700 - val_loss: 39.9169 - val_accuracy: 0.8800
Epoch 7/10
7/7 [==============================] - 0s 27ms/step - loss: 0.4865 - accuracy: 0.8550 - val_loss: 24.0036 - val_accuracy: 0.8800
Epoch 8/10
7/7 [===

In [ ]:
model.save('Classifier/saved_models')

model.summary()

In [ ]:
model = tf.keras.models.load_model('saved_models')

In [ ]:
model = tf.keras.models.load_model('Classifier/saved_models')

In [ ]:

tf.keras.utils.plot_model(model, to_file='model.png',
                          show_shapes=True, show_layer_activations=True)

In [7]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)



In [5]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)


AttributeError: 'function' object has no attribute 'Figure'

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [3]:
def test_model(img_path):
    img = tf.keras.utils.load_img(
        img_path, target_size=(img_height, img_width))

    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(class_names[np.argmax(score)], 100 * np.max(score))
    )

In [ ]:
# Read the image
# img = Image.open('Classifier/mnist_png/testing/0/3.png')
#
# # define a transform to convert the image to grayscale
# transform = transforms.Grayscale()
#
# # apply the above transform on the image
# img = transform(img)
#
# # dispaly the image
# #img.show()
#
# convert_tensor = transforms.ToTensor()
#
# img_tensor = convert_tensor(img)
#
# print(img_tensor.shape)

testing_ds = tf.keras.utils.image_dataset_from_directory(
  pathlib.Path('Classifier/mnist_png/me'),
  seed=123,
  image_size=(img_height, img_width),
  batch_size=1,
  color_mode='grayscale'
  )

testing_normalized_ds = testing_ds.map(lambda x, y: (normalization_layer(x), y))
testing_image_batch, testing_labels_batch = next(iter(testing_normalized_ds))


In [113]:
#print(train_ds[0])
model.predict(testing_ds)

2/2 [==============================] - 0s 5ms/step


array([[ -292.7022  , -1864.0464  ,  1193.3007  ,  1272.931   ,
        -1080.4885  ,   -43.669655,  -760.62317 , -1429.5154  ,
          257.2695  , -2579.77    ],
       [  259.88556 , -1370.9014  ,   270.61807 ,    90.21795 ,
         -976.1065  ,  -378.90308 ,  -185.88264 , -1444.0776  ,
          745.89233 ,  -855.32806 ]], dtype=float32)